In [65]:
import tensorflow.keras as keras

In [74]:
reconstructed_model = keras.models.load_model("my_h5_model1.h5")

In [3]:
import tensorflow as tf



In [5]:
import pickle

In [75]:
file = open("test_data.pkl",'rb')
test_data = pickle.load(file)
file1 = open("test_labels.pkl",'rb')
test_labels = pickle.load(file1)

In [68]:
import numpy as np

In [11]:
reconstructed_model.input_names[0]

'input_3'

In [76]:
input_name = reconstructed_model.input_names[0]
index = reconstructed_model.input_names.index(input_name)
reconstructed_model.inputs[index].set_shape([1, 1960])
#converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

In [47]:
  float_converter = tf.lite.TFLiteConverter.from_keras_model(reconstructed_model)
  float_tflite_model = float_converter.convert()
  float_tflite_model_size = open("FLOAT_MODEL_TFLITE.tflite", "wb").write(float_tflite_model)
  print("Float model is %d bytes" % float_tflite_model_size)

INFO:tensorflow:Assets written to: /tmp/tmpbmyv8h55/assets


INFO:tensorflow:Assets written to: /tmp/tmpbmyv8h55/assets


Float model is 51804 bytes


In [77]:
  float_converter = tf.lite.TFLiteConverter.from_keras_model(reconstructed_model)
  float_tflite_model = float_converter.convert()
  float_tflite_model_size = open("FLOAT_MODEL_TFLITE.tflite", "wb").write(float_tflite_model)
  print("Float model is %d bytes" % float_tflite_model_size)

  converter = tf.lite.TFLiteConverter.from_keras_model(reconstructed_model)
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  converter.inference_input_type = tf.int8
  converter.inference_output_type = tf.int8
  def representative_dataset_gen():
    for i in range(100):
      data = test_data[i] 
      flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
      yield [flattened_data]
  converter.representative_dataset = representative_dataset_gen
  tflite_model = converter.convert()
  tflite_model_size = open("MODEL_TFLITE.tflite", "wb").write(tflite_model)
  print("Quantized model is %d bytes" % tflite_model_size)


INFO:tensorflow:Assets written to: /tmp/tmp27xe7054/assets


INFO:tensorflow:Assets written to: /tmp/tmp27xe7054/assets


Float model is 669716 bytes
INFO:tensorflow:Assets written to: /tmp/tmplqndcaai/assets


INFO:tensorflow:Assets written to: /tmp/tmplqndcaai/assets


Quantized model is 171296 bytes


In [15]:
print(test_data.shape)
print(test_labels.shape)

(1113, 1960)
(1113,)


In [21]:
np.expand_dims(test_data,axis=2).shape

(2213, 1960, 1)

(1960,)

In [78]:
def run_tflite_inference(tflite_model_path, test_data,test_labels,model_type="Float",):
  # Load test data
  np.random.seed(0) # set random seed for reproducible test results.

  #test_data = np.expand_dims(test_data, axis=2).astype(np.float32)
  test_data = np.expand_dims(test_data, axis=1).astype(np.float32)
  print(test_data.shape)
  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(tflite_model_path)
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]
  #print(input_details)
  #print(output_details)

  # For quantized models, manually quantize the input data from float to integer
  if model_type == "Quantized":
    input_scale, input_zero_point = input_details["quantization"]
    test_data = test_data / input_scale + input_zero_point
    test_data = test_data.astype(input_details["dtype"])

  correct_predictions = 0
  for i in range(len(test_data)):
    #print('shape is')
    #print(test_data[i].shape)
    
    interpreter.set_tensor(input_details["index"],test_data[i])
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    top_prediction = output.argmax()
    correct_predictions += (top_prediction == test_labels[i])

  print('%s model accuracy is %f%% (Number of test samples=%d)' % (
      model_type, (correct_predictions * 100) / len(test_data), len(test_data)))




In [80]:
run_tflite_inference("FLOAT_MODEL_TFLITE.tflite",test_data,test_labels,model_type='Float')

# Compute quantized model accuracy
run_tflite_inference("MODEL_TFLITE.tflite",test_data,test_labels, model_type='Quantized')

(3291, 1, 1960)
Float model accuracy is 79.489517% (Number of test samples=3291)
(3291, 1, 1960)
Quantized model accuracy is 79.367973% (Number of test samples=3291)


In [42]:
import os

In [43]:
MODEL_TFLITE = os.path.join('./', 'MODEL_TFLITE.tflite')
#FLOAT_MODEL_TFLITE = os.path.join(MODELS_DIR, 'float_model.tflite')
MODEL_TFLITE_MICRO = os.path.join('./', 'model.cc')

In [73]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}
# Update variable names
REPLACE_TEXT = MODEL_TFLITE.replace('/', '_').replace('.', '_')
!sed -i 's/'{REPLACE_TEXT}'/g_model/g' {MODEL_TFLITE_MICRO}

In [25]:
with open('test_data.pkl','wb') as file:
   pickle.dump(test_data,file)

In [26]:
with open('test_labels.pkl','wb') as file:
   pickle.dump(test_labels,file)

In [13]:

converter = tf.lite.TFLiteConverter.from_keras_model(reconstructed_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
def representative_dataset_gen():
 for i in range(100):
  data, _ = audio_processor.get_data(1, i*1, model_settings,
                                     BACKGROUND_FREQUENCY, 
                                     BACKGROUND_VOLUME_RANGE,
                                     TIME_SHIFT_MS,
                                     'testing',
                                     sess)
  flattened_data = np.array(data.flatten(), dtype=np.float32).reshape(1, 1960)
  yield [flattened_data]
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()
tflite_model_size = open(MODEL_TFLITE, "wb").write(tflite_model)
print("Quantized model is %d bytes" % tflite_model_size)


NotImplementedError: numpy() is only available when eager execution is enabled.